In [ ]:
import sys

sys.path.append("..")

In [ ]:
from main import OCRModel
from inference import Inference

In [ ]:
checkpoint = ""

In [ ]:
inference = Inference(image_path="inference_image/abc/paper_img_002844.png")
predict_dataloader = inference.predict_dataloader()
model = OCRModel.load_from_checkpoint(
    checkpoint, strict=True, batch_max_length=200, dali=True, map_location="cuda", pred_name="attn"
)
model.eval()

for batch in predict_dataloader:
    preds = model.predict_step(batch, 0)
    print(preds)